# Load required libs

In [1]:
%cd ..
%pwd
import sys
sys.path.append('./src')
from src.data_processor import RawDataProcessor
from src.problem_config import create_prob_config
from src.drift_detector import ks_drift_detect
prob_config = create_prob_config("phase-2", "prob-2")

c:\VENV\api_prediction


In [2]:
from mlflow.models.signature import infer_signature
import mlflow

def log_model_to_tracker_lgbm(model, metrics, desc):
    MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment("phase-2_prob-2_lgbm")
    MLFLOW_MODEL_PREFIX = "model"
    mlflow.start_run(description=desc)
    mlflow.log_metrics(metrics)
    mlflow.log_params(model.get_params())
    signature = infer_signature(test_x.astype(np.float64), predictions)
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=MLFLOW_MODEL_PREFIX,
        signature=signature,
        pip_requirements ='src/requirements.txt'
        #registered_model_name="phase-1_prob-1_model-1"
    )

    experimentid = mlflow.active_run().info.run_id
    mlflow.end_run()
    return experimentid

# Prepare datasets

In [146]:
# Import xtran and ytrain
import pandas as pd
import numpy as np

training_data = pd.read_parquet(prob_config.raw_data_path)

training_data, category_index = RawDataProcessor.build_category_features(
            training_data, prob_config.categorical_cols
        )

target_col = prob_config.target_col
train_x0 = training_data.drop([target_col], axis=1)
train_y0 = training_data[[target_col]]

import pickle
with open(prob_config.category_index_path, "wb") as f:
    pickle.dump(category_index, f)

In [145]:
from sklearn.model_selection import train_test_split

train, dev = train_test_split(training_data, test_size=0.1, random_state=123)

train_x = train.drop(["label"], axis=1)
train_y = train[[target_col]]
test_x = dev.drop(["label"], axis=1)
test_y = dev[[target_col]]

In [186]:
from lightgbm import LGBMClassifier

#eval_set = [(train_x, train_y), (test_x, test_y)]
model0 = LGBMClassifier(objective="multiclass", random_state=123, n_estimators=100)
model0.fit(train_x, train_y, verbose=False) #eval_set=eval_set,

c:\VENV\api_prediction\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\VENV\api_prediction\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\VENV\api_prediction\.venv\Lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier(objective='multiclass', random_state=123)

In [193]:
from sklearn.metrics import accuracy_score
predictions = model0.predict(test_x)
#predictions = s.predict_model(best[0], data = test_x)["prediction_label"]
accuracy = accuracy_score(predictions, test_y)
metrics = {"accuracy_score": accuracy}
print(f"metrics: {metrics}")

metrics: {'accuracy_score': 0.8223120452708165}


In [191]:
%%timeit -n 10
model0.predict_proba(test_x)

60.5 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [153]:
import lleaves
model_path = ".venv/phase2_2_lgbm.txt"
model0.booster_.save_model(filename=model_path)
llvm_model = lleaves.Model(model_file=model_path)
llvm_model.compile()

In [192]:
%%timeit -n 10
z = llvm_model.predict(test_x)
labels = np.argmax(z, axis=1)
classes = model0.classes_
labels = [classes[i] for i in labels]

16 ms ± 360 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
run_description = """
### Header
LGBM model, First Base Model Prob2
Model: LGBM
    """
log_model_to_tracker_lgbm(model0, metrics, run_description)

c:\VENV\api_prediction\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


'6b1fc425890a4574a352aa72de7ffb3f'

# Drift Detection 

In [34]:
# Save referent for drift detection.
#X_baseline_df = pd.DataFrame(X_baseline, columns=col_to_detect)
#X_baseline_df.to_parquet(prob_config.driff_ref_path, index=False)

# Model drift

In [1]:
import pandas as pd
import redis
import pickle
# Load data for problem 1
rc2 = redis.Redis(host='192.168.88.113', db=2, port=6379)

captured_x = pd.DataFrame()
for key in rc2.keys():
    captured_data = pickle.loads(rc2.get(key))
    captured_x = pd.concat([captured_x, captured_data])

captured_x.drop_duplicates(inplace=True, ignore_index=True)

#captured_x = apply_category_features(
#    raw_df=captured_x[train_x0.columns],
#    categorical_cols=prob_config.categorical_cols,
#    category_index=category_index,
#)

In [ ]:
np_captured_x = captured_x.copy() #drop(["is_drift", "batch_id"], axis=1)
np_captured_x = np_captured_x.astype(train_x.dtypes.to_dict())
np_captured_x['label'].fillna(-1, inplace=True)

In [2]:
allkey = rc2.keys()